# Image Detection Demo with PytorchWildlife

This tutorial guides you on how to use PyTorchWildlife for image detection. We will go through the process of setting up the environment, defining the detection model, as well as performing inference and saving the results in different ways.

## Prerequisites
Install PytorchWildlife running the following commands:
```bash
conda create -n pytorch_wildlife python=3.8 -y
conda activate pytorch_wildlife
pip install PytorchWildlife
```
Also, make sure you have a CUDA-capable GPU if you intend to run the model on a GPU. This notebook can also run on CPU.

## Importing libraries
First, we'll start by importing the necessary libraries and modules.

In [ ]:
import os
import torch
from PytorchWildlife.models import detection as pw_detection
from PytorchWildlife import utils as pw_utils

## Setting Device
If you are using a GPU for this exercise, please specify which GPU to use for the computations. By default, GPU number 0 is used.

In [3]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
if DEVICE == "cuda":
    torch.cuda.set_device(0)

## Model Initialization
We will initialize the MegaDetectorV6 model for image detection. This model is designed for detecting animals in images.

In [ ]:
detection_model = pw_detection.MegaDetectorV6(device=DEVICE, pretrained=True, version="yolov9c")

If you want to use MegaDetectorV5, run this line.

In [ ]:
detection_model = pw_detection.MegaDetectorV5(device=DEVICE, pretrained=True, version="a")

## Single Image Detection
We will first perform detection on a single image. Make sure to verify that you have the image in the specified path.

In [6]:
tgt_img_path = os.path.join(".", "demo_data", "imgs", "10050028_0.JPG")
results = detection_model.single_image_detection(tgt_img_path)
pw_utils.save_detection_images(results, os.path.join(demo_path,"demo_output"), overwrite=False)
pw_utils.save_crop_images(results, os.path.join(".","crop_output"), overwrite=False)

## Batch Image Detection
Next, we'll demonstrate how to process multiple images in batches. This is useful when you have a large number of images and want to process them efficiently.

In [ ]:
tgt_folder_path = os.path.join(".", "demo_data", "imgs")
results = detection_model.batch_image_detection(tgt_folder_path, batch_size=16, extension="JPG")

## Output Results
PytorchWildlife allows to output detection results in multiple formats. Here are the examples:

### 1. Annotated Images:
This will output the images with bounding boxes drawn around the detected animals. The images will be saved in the specified output directory.

In [8]:
pw_utils.save_detection_images(results, "batch_output", overwrite=False)

### 2. Cropped Images:
This will output the cropped images of the detected animals. The cropping is done around the detection bounding box, The images will be saved in the specified output directory.

In [9]:
pw_utils.save_crop_images(results, "crop_output", overwrite=False)

### 3. JSON Format:
This will output the detection results in JSON format. The JSON file will be saved in the specified output directory.

In [10]:
pw_utils.save_detection_json(results, os.path.join(".","batch_output.json"),
                             categories=detection_model.CLASS_NAMES,
                             exclude_category_ids=[], # Category IDs can be found in the definition of each model.
                             exclude_file_path=None)

### 3.1 Timelapse JSON Format:
PytorchWildlife is compatible with Timelapse, a popular tool to visualize and process camera trap data. This code will output the detection results in a JSON format compatible with Timelapse:

In [11]:
pw_utils.save_detection_timelapse_json(results, os.path.join(demo_path,"batch_output_timelapse.json"),
                                       categories=detection_model.CLASS_NAMES,
                                       exclude_category_ids=[], # Category IDs can be found in the definition of each model.
                                       exclude_file_path=tgt_folder_path,
                                       info={"detector": "MegaDetectorV6"})

### Copyright (c) Microsoft Corporation. All rights reserved.
### Licensed under the MIT License.